# Laboratory 1

Import important libraries

In [1]:
import copy
import random
import numpy as np
import matplotlib.pyplot as plt

# Loading data

Dataset, actualy flow and distances matrix can be found on http://anjos.mgi.polymtl.ca/qaplib/inst.html#HRW website. Data in file are saved in the following order *n_facilities, flow_matrix, distance_matrix*.

In [2]:
data_path = "./data/had12.dat"
data_file = open(data_path, 'r')
n_facilities = int(data_file.readline().lstrip().rstrip())
data_file.close()

data_matrix = np.loadtxt(data_path, skiprows=2)
flow_matrix = data_matrix[:n_facilities]
distance_matrix = data_matrix[n_facilities:]

# Quadratic assigment problem

The **quadratic assignment problem** (*QAP*) is one of the fundamental combinatorial optimization problems in the branch of optimization or operations research in mathematics, from the category of the facilities location problems.

The problem models the following real-life problem:

There are a **set of n facilities** and a **set of n locations**. For each pair of locations, a **distance** is specified and for each pair of facilities a **weight** or **flow** is specified (*e.g., the amount of supplies transported between the two facilities*). The problem is to *assign all facilities to different locations with the goal of minimizing the sum of the distances multiplied by the corresponding flows*.
Intuitively, the cost function encourages factories with high flows between each other to be placed close together. <br><br>
More infor can be found on: https://neos-guide.org/content/quadratic-assignment-problem

**COST Function**

![title](img/cost.png)

# Genetic Algorithm

## 1. Introduction

A genetic algorithm is a search heuristic that is inspired by Charles Darwin’s theory of natural evolution. This algorithm reflects the process of natural selection where the fittest individuals are selected for reproduction in order to produce offspring of the next generation.

Popular terms in Genetic algorithm: 
* Population - set of individuals,
* Genes - set of parameters (variables/features),
* Chromosome - solution,

![title](img/terms.png)

## 2. Genetic algorithm steps
Genetic algorithm can be described in few very important steps, each of them is derived from evolution theory. 
Whole algorithm can be implemented in following steps:<br><br>

* START
* Generate the initial population
* Compute fitness
* REPEAT
    * Selection
    * Crossover
    * Mutation
    * Compute fitness
* UNTIL population has converged
* STOP


![title](img/states.png)

### Set initial parameters

Set initial parameters for genetic algorithm. The most important hyperparameters are population_size, crossover_proobability and mutation_probability

In [48]:
population_size = 1000
crossover_probability = 0.8
mutation_probability = 0.008

### Generate random population
In first step Population (set of solutions is randomly generated)

In [59]:
def generate_random_population(n_facilities, n_chromosomes):
    population_list = []
    for ch_idx in range(n_chromosomes):
        rand_chromosome = list(range(0, n_facilities))
        random.shuffle(rand_chromosome)
        population_list.append(rand_chromosome)
    return population_list

### Compute fitness

Fitness function is the most important element in implementing genetic algorithm. Best chromosomes will be choosen according to this function, so it is significant to choose it correctly. It gives a fitness score to each individual. The probability that an individual will be selected for reproduction is based on its fitness score.

In [60]:
def fitness_score(population_list, flow_matrix, distance_matrix):
    n_facilities = len(population_list[0])
    fitness_scores_list = []
    for chromosome_list in population_list: 
        chromosome_fitness = 0
        for loc_idx_f in range(0, n_facilities):
            fac_und_loc_f =  chromosome_list[loc_idx_f]
            for loc_idx_s in range(0, n_facilities):
                fac_und_loc_s =  chromosome_list[loc_idx_s]
                ft_s = flow_matrix[fac_und_loc_f, fac_und_loc_s] * distance_matrix[loc_idx_f, loc_idx_s]
                chromosome_fitness += ft_s
        fitness_scores_list.append(1. / (chromosome_fitness / 2.))
    return fitness_scores_list
print(1. / fitness_score([[4, 3, 6, 0, 1, 2, 5]], flow_matrix, distance_matrix)[0])

213.0


There are few requirements for fitness score, first of them tell that fitness function can't be negative and second tells that results should sum to one.

In [61]:
def normalise_fitness_score(fitness_score):
    return np.array(fitness_score) / np.sum(fitness_score)

### Selection

To select the best individuals from the population, the selection method is used. In selection process few variation can be met like **roulette selection**, **tournament selection**, **elitism**. 

#### Roulette selection

Roulette selection uses mechanism very similiar to roulette wheel in gambling. Actualy it uses cumulative distribution to select appropriate chromosomes. This method is very similiar to inverse-transform methods used for sampling in Machine Learning.

![title](img/roulette.png)

In [78]:
def roulette_selection(population_list, fitness_scores_list, elitism=True):
    new_species = []
    population_size = len(fitness_scores_list)
    population_size = population_size - 1 if elitism else population_size
    cum_sum = np.cumsum(fitness_scores_list, axis=0)

    for _ in range(0, population_size):
#         rnd = random.uniform(0, 1)
#         # for others
#         counter = 0
#         while rnd >= cum_sum[counter]:
#             counter += 1
        new_species.append(population_list[select(fitness_scores_list)])
#         new_species.append(population_list[counter])
    new_species.append(population_list[np.argmax(fitness_scores_list)])
    return new_species

def select (fitnesses):
    """Select a value from the array fitnesses on a random number
    based on the fitness values."""
    selection = random.random() * sum(fitnesses)

    index = 0;
    while selection > 0:
        selection -= fitnesses[index]
        index += 1
    return index - 1

print(roulette_selection([1, 2, 3, 4, 5, 6, 7], [0.4, 0, 0.05, 0.05, 0.1, 0, 0.4]))

[7, 7, 1, 7, 7, 4, 1]


#### Tournament selection

**Tournament selection** is a method of selecting an individual from a population of individuals in a genetic algorithm. Tournament selection involves **running several "tournaments"** among a few individuals (or "chromosomes") chosen at random from the population. **The winner of each tournament (the one with the best fitness) is selected for crossover**.

* Choose few individuals at random from the population (a tournament).
* The individual with the best fitness (the winner) is selected for crossover.

![title](img/tournament.jpg)

In [53]:
def tournament_selection(population_list, fitness_scores_list, elitism=True):
    # Create new population
    new_species = []
    population_size = len(fitness_scores_list)
    population_size = population_size - 1 if elitism else population_size
    for _ in range(0, population_size):
        # Take best
        of_parent_idx = random.randint(0, len(fitness_scores_list) - 1)
        tf_parent_idx = random.randint(0, len(fitness_scores_list) - 1)
        if fitness_scores_list[of_parent_idx] > fitness_scores_list[tf_parent_idx]:
            ch_winner = population_list[of_parent_idx]
        else:
            ch_winner = population_list[tf_parent_idx]
        new_species.append(ch_winner)
    if elitism:
        new_species.append(population_list[np.argmax(fitness_scores_list)])
    return new_species

### Crossover

Crossover is the most significant phase in a genetic algorithm. For each pair of parents to be mated, a crossover point is chosen at random from within the genes.

For example, consider the crossover point to be 3 as shown below.

![title](img/crossover.png)

Offspring are created by exchanging the genes of parents among themselves until the crossover point is reached.

![title](img/corssover_2.png)

The new offspring are added to the population.

![title](img/crossover_3.png)

In [54]:
def chromosome_crossover(chromosome_o, chromosome_s):
    # Random point to crossover
    chr_o, chr_s = copy.copy(chromosome_o), copy.copy(chromosome_s)
    pos = random.randint(0, len(chromosome_o) - 1)

    # Change chromosome
    for ch_idx in range(0, pos):
        # values on ind
        fac_o = chr_o[ch_idx]
        fac_s = chr_s[ch_idx]
        
        # Values for swap
        fac_os_idx = chr_o.index(fac_s)
        fac_so_idx = chr_s.index(fac_o)
        
        # Save values
        chr_o[fac_os_idx] = fac_o
        chr_s[fac_so_idx] = fac_s
        
        # Change values
        chr_o[ch_idx] = fac_s
        chr_s[ch_idx] = fac_o
    return chr_o, chr_s

In [55]:
def crossover_population(new_species, crossover_probability):
    # Select for crossover
    species_nc = []
    crossover_list = []
    for n_chrom in new_species:
        rnd = random.uniform(0, 1)
        if rnd < crossover_probability:
            crossover_list.append(n_chrom)
        else:
            species_nc.append(n_chrom)
            
    crossover_tuples = []    
    # Create crossover buddies
    cr_iterate = list(enumerate(crossover_list))
    while cr_iterate:
        cch_idx, c_chrom = cr_iterate.pop()
        if not cr_iterate:
            species_nc.append(c_chrom)
            break
        cb_idx, cross_buddy = random.choice(cr_iterate)
        cr_iterate = [(x_k, x_v) for x_k, x_v in cr_iterate if x_k != cb_idx]
        crossover_tuples.append((c_chrom, cross_buddy))

    # Crossover to list
    after_cover = []
    for cr_tup in crossover_tuples:
        cr_o, cr_t = chromosome_crossover(cr_tup[0], cr_tup[1])
        after_cover.append(cr_o)
        after_cover.append(cr_t)

    # New population
    new_species = after_cover + species_nc
    return new_species
# new_species = [[7, 4, 3, 2, 5, 6, 1, 8], [4, 6, 5, 2, 1, 8, 7, 3]]
# print(crossover_population(new_species, 1.0))

### Mutation

Mutation is a genetic operator used to maintain genetic diversity from one generation of a population of genetic algorithm chromosomes to the next. It is analogous to biological mutation. Mutation alters one or more gene values in a chromosome from its initial state. In mutation, the solution may change entirely from the previous solution. Hence GA can come to a better solution by using mutation. Mutation occurs during evolution according to a user-definable mutation probability. This probability should be set low. If it is set too high, the search will turn into a primitive random search.

![title](img/mutation.png)

In TSP and QAP problem mutation will have slightly different form. We will choose two genes and swap them.

In [56]:
def mutation_population(new_species, mutation_probability):
    # # MUTATION
    mutated = []
    for chromosome in new_species:
        for b_idx in range(0, len(chromosome)):
            rnd = random.uniform(0, 1)
            if rnd < mutation_probability:
                swap_idx = random.randint(0, len(chromosome) - 1)
                old_mut_val = chromosome[b_idx]
                chromosome[b_idx] = chromosome[swap_idx]
                chromosome[swap_idx] = old_mut_val
        mutated.append(chromosome)
    return mutated

### Concatenate all steps

In [79]:
population_list = generate_random_population(n_facilities, population_size)
for epoch in range(0, 100000):
    fit_scores = fitness_score(population_list, flow_matrix, distance_matrix)
    fit_scores_norm = normalise_fitness_score(fit_scores)
    selected_ch = roulette_selection(population_list, fit_scores_norm, elitism=True)
#     selected_ch = tournament_selection(population_list, fit_scores_norm, elitism=True)
    crossed_ch = crossover_population(selected_ch, crossover_probability)
    mutated_ch = mutation_population(crossed_ch, mutation_probability)
    max_fitness = np.max(fit_scores)
    max_chromosome = [x + 1 for x in population_list[np.argmax(fit_scores)]]
    print("Epoch: {}, Population fitness score: {}, Max score: {}, Max chromosome: {}".format(epoch, 1. / np.mean(fit_scores), 
                                                                                              1. / max_fitness, max_chromosome))
    population_list = mutated_ch

Epoch: 0, Population fitness score: 944.062913812, Max score: 861.0, Max chromosome: [3, 10, 2, 5, 7, 12, 1, 11, 8, 6, 9, 4]
Epoch: 1, Population fitness score: 941.602964363, Max score: 864.0, Max chromosome: [4, 5, 9, 12, 7, 2, 6, 11, 1, 3, 10, 8]
Epoch: 2, Population fitness score: 942.058123867, Max score: 870.0, Max chromosome: [8, 2, 6, 3, 7, 10, 11, 5, 1, 12, 4, 9]
Epoch: 3, Population fitness score: 942.339107118, Max score: 867.0, Max chromosome: [9, 4, 5, 6, 11, 7, 1, 10, 8, 12, 2, 3]
Epoch: 4, Population fitness score: 942.400515687, Max score: 863.0, Max chromosome: [10, 8, 11, 2, 12, 5, 6, 7, 3, 9, 1, 4]
Epoch: 5, Population fitness score: 941.100899393, Max score: 858.0, Max chromosome: [4, 1, 11, 6, 7, 12, 2, 10, 9, 5, 3, 8]
Epoch: 6, Population fitness score: 943.732102687, Max score: 857.0, Max chromosome: [4, 1, 11, 6, 7, 12, 2, 10, 9, 5, 3, 8]
Epoch: 7, Population fitness score: 943.189016126, Max score: 861.0, Max chromosome: [8, 3, 2, 6, 4, 7, 12, 1, 5, 11, 10, 9]


Epoch: 66, Population fitness score: 943.094786647, Max score: 874.0, Max chromosome: [6, 3, 12, 11, 5, 8, 7, 4, 10, 2, 9, 1]
Epoch: 67, Population fitness score: 942.799075546, Max score: 862.0, Max chromosome: [3, 8, 12, 7, 2, 6, 5, 10, 11, 1, 9, 4]
Epoch: 68, Population fitness score: 943.057049201, Max score: 859.0, Max chromosome: [4, 9, 11, 1, 12, 5, 7, 6, 10, 3, 2, 8]
Epoch: 69, Population fitness score: 943.747543268, Max score: 875.0, Max chromosome: [5, 10, 2, 3, 6, 12, 7, 11, 8, 1, 9, 4]
Epoch: 70, Population fitness score: 943.029402631, Max score: 869.0, Max chromosome: [3, 7, 6, 2, 1, 11, 10, 5, 8, 12, 4, 9]
Epoch: 71, Population fitness score: 941.212261551, Max score: 859.0, Max chromosome: [3, 2, 10, 8, 7, 5, 6, 11, 12, 1, 4, 9]
Epoch: 72, Population fitness score: 942.176079536, Max score: 867.0, Max chromosome: [5, 12, 3, 9, 11, 1, 7, 2, 6, 4, 10, 8]
Epoch: 73, Population fitness score: 943.495524588, Max score: 869.0, Max chromosome: [8, 10, 5, 6, 3, 11, 12, 1, 2, 7

KeyboardInterrupt: 